In [79]:
from skimage import io, transform, img_as_ubyte
import io as iosys
import requests
import sys
import os

In [80]:
def read_image(route):
    try:
        if route.startswith("http://") or route.startswith("https://"):
            # Leer la imagen desde una URL
            response = requests.get(route)
            if response.status_code == 200:
                imagen = io.imread(route)
                return imagen
            else:
                print(f"Error al descargar la imagen. Código de respuesta: {response.status_code}")
                return None
        else:
            # Leer la imagen desde un archivo temporal local
            with open(route, 'rb') as f:
                imagen = io.imread(BytesIO(f.read()))
                return imagen
    except Exception as e:
        print(f"Error al leer la imagen: {e}")
        return None

In [99]:
def recursive_image_resizing(image, format, new_dimension, max_size, current_depth=0, max_depth=5):
    try:
        original_width, original_height = image.shape[:2]
        new_height = int((new_dimension / original_width) * original_height)
        
        # Redimensionar la imagen
        resized_image = transform.resize(image, (new_dimension, new_height), anti_aliasing=True)
        # Convertir la imagen a tipo uint8
        resized_image_uint8 = img_as_ubyte(resized_image)
        
        buffer = iosys.BytesIO()
        io.imsave(buffer, resized_image_uint8, format=format)
        
        if buffer.tell() > (max_size * 1024):
            if current_depth < max_depth:
                return recursive_image_resizing(resized_image, format, int(new_dimension * .75), max_size, current_depth + 1)
        return resized_image_uint8
    except Exception as e:
        print(f"Error resizing image: {e}")
        return None

In [100]:
def see_image(image):
    # Mostrar la imagen redimensionada
    io.imshow(image)
    io.show()

In [101]:
image = read_image('https://d3t3ozftmdmh3i.cloudfront.net/production/podcast_uploaded_nologo/35411244/35411244-1682544213802-5018213ca7789.jpg')
resize_image = recursive_image_resizing(image, 'jpeg', 900, 300)
io.imsave('test-skimage.jpeg', resize_image)